In [1]:
import pandas as pd
import openpyxl
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill, Color
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from openpyxl import load_workbook

pd.set_option('future.no_silent_downcasting', True)


#### Utilities Functions

In [2]:
color_dict = {
    'Y': 'FFFF00',  # Yellow
    'G': '00FF00',  # Green
    'L': '0000FF',  # Blue
    'R': 'FF0000',  # Red
    'W': 'FFFFFF',  # White
    'LG': '808080', # Gray (Light Gray)
    'O': 'FFA500',  # Orange
    'BR': '80471c',#'A52A2A', # Brown
    'V': '800080',#'8A2BE2',   # Violet
    'GY': '707070',#'C0C0C0',  # Gray
    'B': '000000' ,  # Black (Noir)
    'P' : 'FFC0CB', # Pink
    'C' : '00FFFF', # Cyan
    'D' : 'FFFFF0', # Ivory
    'SI' : '505050', # Silver
}

def apply_color(symbol, cell):
    if '/' in str(symbol):
        primary, secondary = symbol.split('/')
        fill = PatternFill(start_color=color_dict[primary], end_color=color_dict[primary], fill_type="solid")
        cell.fill = fill
        
        side = Side(border_style="thick", color=color_dict[secondary])
        border = Border(diagonal=side, diagonalUp=True)
        cell.border = border
    elif symbol in color_dict:
        fill = PatternFill(start_color=color_dict[symbol], end_color=color_dict[symbol], fill_type="solid")
        cell.fill = fill
    else :
        cell.value = symbol
    

In [3]:
# def validate_colors_in_dataset():
#     unique_colors = df_input['Colour'].dropna().unique()
    
#     unknown_colors = {}
#     for color in unique_colors:
        
#         if '/' in color:
#             primary, secondary = color.split('/')
#             if primary not in color_dict:
#                 if primary in unknown_colors:
#                     unknown_colors[primary] += df_input['Colour'].eq(color).sum()
#                 else:
#                     unknown_colors[primary] = df_input['Colour'].eq(color).sum()
#             if secondary not in color_dict:
#                 if secondary in unknown_colors:
#                     unknown_colors[secondary] += df_input['Colour'].eq(color).sum()
#                 else:
#                     unknown_colors[secondary] = df_input['Colour'].eq(color).sum()
#         else:
#             if color not in color_dict:
#                 if color in unknown_colors:
#                     unknown_colors[color] += df_input['Colour'].eq(color).sum()
#                 else:
#                     unknown_colors[color] = df_input['Colour'].eq(color).sum()
    
#     if unknown_colors:
#         print(f"Found {len(unknown_colors)} unknown color codes:")
#         for color, count in unknown_colors.items():
#             print(f"  - '{color}': appears {count} times")
#         print("These colors will not be correctly displayed in the Excel file.")
#     else:
#         print("All colors in the dataset are defined in color_dict.")
    
#     # Display color distribution
#     print("\nColor distribution in dataset:")
#     color_counts = df_input['Colour'].value_counts().head(10)
#     print(color_counts)
    
#     return unknown_colors

In [4]:
input_file = r'C:\Users\user\Desktop\YMM-2\Data\Input\G60 WI.xlsx'
# output_file = r'..\Data\Output\Schema G60.xlsx'

In [5]:
df_input = pd.read_excel(input_file)

In [6]:
df_input['Colour'].value_counts()

Colour
BR     1243
G       564
R       462
W       424
G/L     415
       ... 
TA        2
0.5       1
10        1
0,5       1
211       1
Name: count, Length: 97, dtype: int64

In [7]:
all_sheets = list(df_input['SPS'].unique())

In [8]:
PM_sps = list(df_input['Production Module MA15'].unique())

#### Create a new workbook

In [9]:
path_output_excel = 'Final_02_04.xlsx'
# wb = openpyxl.Workbook()
# ws = wb.active

#### Yazaki header template

In [10]:
def header_final_schema(ws):
    # wb = load_workbook(file_path)
    # ws = wb.active 
    column_widths = [15, 20, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
    for i, width in enumerate(column_widths, 1):
        ws.column_dimensions[get_column_letter(i)].width = width
    
    thin_border = Border(
        left=Side(style='medium'), 
        right=Side(style='medium'), 
        top=Side(style='medium'), 
        bottom=Side(style='medium')
    )
    
    # Header row 1 - Yazaki logo area
    img = Image('../yazaki_logo.png')
    cell_logo = ws.merge_cells('A1:J1')
    ws.add_image(img, cell_logo)
    # ws['A1'] = 'YAZAKI'
    # ws['A1'].font = Font(name='Arial', size=14, bold=True)
    # ws['A1'].alignment = Alignment(horizontal='left', vertical='center')
    
    blue_fill = PatternFill(start_color='0070C0', end_color='0070C0', fill_type='solid')
    blue_font = Font(name='Arial', size=10, color=Color(rgb='FFFFFF'))
    
    ws.merge_cells('K1:O1')
    first_cell = ws.cell(row=1, column=11)  
    first_cell.value = 'PU24/PU25 LHD'
    first_cell.font = blue_font
    first_cell.fill = blue_fill
    first_cell.alignment = Alignment(horizontal='center', vertical='center')
    
    header_fill = PatternFill(start_color='D9D9D9', end_color='D9D9D9', fill_type='solid')
    header_font = Font(name='Arial', size=10)
    
    cell_configs = [
        ('C2', 'Nº du Produit / Niveau', header_font, header_fill, thin_border),
        ('C3', 'KAR G60', Font(name='Arial', size=10), None, thin_border),
        ('E2', 'Local du travail', header_font, header_fill, thin_border),
        ('E3', 'WP414-SA513', Font(name='Arial', size=10), None, thin_border),
        ('F2', 'N° de ligne', header_font, header_fill, thin_border),
        ('F3', '1', Font(name='Arial', size=10), None, thin_border),
        ('G2', 'N° de ligne', header_font, header_fill, thin_border),
        ('G3', '1', Font(name='Arial', size=10), None, thin_border),
        ('H2', 'N° de ligne', header_font, header_fill, thin_border),
        ('H3', '1', Font(name='Arial', size=10), None, thin_border),
        ('I2:J2', 'Processus', header_font, header_fill, thin_border),
        ('I3:J3', 'C2', Font(name='Arial', size=10), None, thin_border),
        ('M2', 'Nº de Registre', header_font, header_fill, thin_border),
        ('M3', 'EA-EN-MMO-xx-T-6047', Font(name='Arial', size=10), None, thin_border)
    ]
    
    for cell_range, value, font, fill, border in cell_configs:
        if ':' in cell_range:
            ws.merge_cells(cell_range)
            # Get the first cell of the merged range
            first_cell = ws[cell_range.split(':')[0]]
            first_cell.value = value
            first_cell.font = font
            
            if fill:
                first_cell.fill = fill
            
            first_cell.border = border
            first_cell.alignment = Alignment(horizontal='center', vertical='center')
        else:
            cell = ws[cell_range]
            cell.value = value
            cell.font = font
            
            if fill:
                cell.fill = fill
            
            cell.border = border
            cell.alignment = Alignment(horizontal='center', vertical='center')
    
    # Add borders to entire merged ranges
    border_ranges = ['C2:D2', 'I2:J2', 'M2:O2', 'C3:D3', 'I3:J3', 'M3:O3']
    for cell_range in border_ranges:
        for row in ws[cell_range]:
            for cell in row:
                cell.border = thin_border
    
    # print("Yazaki header template created successfully!")
    return ws

path = r"C:\Users\user\Desktop\Output ymm2\new.xlsx"
wb = load_workbook(path)
# ws = wb.active 

for ws in wb.worksheets:
    ws = header_final_schema(ws)
    
# header_final_schema(path)
wb.save(path)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\user\\Desktop\\Output ymm2\\new.xlsx'

#### Final Schema

In [16]:
import xlwings as xw
import pandas as pd
import numpy as np

# all_sheets = ['SA36-A', 'SA47', 'SA513'] 

def schema_final_without_color(path_output_excel)  :
    with pd.ExcelWriter(path_output_excel, engine='xlsxwriter') as writer:      
        for sheet in all_sheets:
            if pd.isna(sheet):
                continue
            
            sps_data = df_input[df_input['SPS'] == sheet]
            
            if sps_data.empty:
                continue
            
            pm = sps_data['Production Module MA15'].fillna('').astype(str)
            step = sps_data['Step'].fillna('').infer_objects(copy=False).astype(str)
            
            arrays = [np.array(pm), np.array(step)]
            index = pd.MultiIndex.from_arrays(arrays, names=('Production Module MA15', 'Sequence'))
            
            # Création du DataFrame avec la colonne renommée
            df = pd.DataFrame({'Materiel': sps_data['Component Name'].values,
                            'SAP NO MA15': sps_data['SAP NO MA15'].values,
                            'Note': sps_data['Note'].values,
                            'CS' : sps_data['CS'].values,
                            'color' : sps_data['Colour'].values,
                            '' : '',
                            'CON A' : sps_data['From Connector'].values,
                            'CAV A' : sps_data['From Cavity'].values,
                            'INSERTION A' : '',
                            'CON B' : sps_data['To Connector'].values,
                            'CAV B' : sps_data['To Cavity'].values,
                            'INSERTION B' : '',
                            },
                            index=index)
            df.to_excel(writer, sheet_name=sheet, startrow=4,startcol=0,index=True)
    print('Schema without color created successfully!')

schema_final_without_color(path_output_excel)

Schema without color created successfully!


In [17]:
def add_unique_ids_to_schema(wb,sheet_name):
    
    ws = wb[sheet_name]
        
    header_row = 5
        
    ws.insert_cols(1)
    id_cell = ws.cell(row=header_row, column=1)
    id_cell.value = "Séquence PM"
    id_cell.font = Font(bold=True)
    id_cell.alignment = Alignment(horizontal='center', vertical='center')
        
    id_counter = 1
    for row in range(header_row + 1, ws.max_row + 1):
        has_content = any(ws.cell(row=row, column=col).value for col in range(2, ws.max_column + 1))
        if has_content:
            id_cell = ws.cell(row=row, column=1)
            id_cell.value = id_counter
            id_cell.alignment = Alignment(horizontal='center', vertical='center')
            id_counter += 1
    

    return ws


#### Final schema with color

In [12]:
# Replace '/' with a space in the sheet name
sheet_name = 'sa276//sa279'
sheet_name = sheet_name.replace('/', ' ')
print(sheet_name)  # Output: 'sa276 sa279'

sa276  sa279


In [18]:
def schema_final_with_color(wb_path):
    wb = load_workbook(wb_path)

    for sheet_name in wb.sheetnames:
        
        sheet = add_unique_ids_to_schema(wb,sheet_name)
        
        color_col = None
        header_row = 5
        
        for col in range(1, sheet.max_column + 1):
            if sheet.cell(row=header_row, column=col).value == "color":
                color_col = col
                break
        
        if color_col is None:
            print(f"No color column found in sheet: {sheet_name}")
            continue
        
        # Add new X column header after last column
        # x_col = sheet.max_column + 1
        # sheet.cell(row=header_row, column=x_col).value = "X"
        
        for row in range(header_row + 1, sheet.max_row + 1):
            color_symbol = sheet.cell(row=row, column=color_col).value
            if color_symbol:
                cell = sheet.cell(row=row, column=color_col + 1)
                try:
                    apply_color(color_symbol, cell)
                except KeyError:
                    print(f"Unknown color code: {color_symbol} in sheet {sheet_name}, row {row}")
                    cell.value = "Unknown color code" #PatternFill(start_color="C0C0C0", end_color="C0C0C0", fill_type="solid")
    print("Color formatting completed successfully.")
    wb.save(path_output_excel)
# wb = schema_final_with_color(path_output_excel)
schema_final_with_color(path_output_excel)

Color formatting completed successfully.


#### Add last table

In [19]:
def add_last_table(file_path):
    wb = load_workbook(file_path)
    # ws = wb.active 
    for ws in wb.worksheets:
        ws = header_final_schema(ws)
        row_start = ws.max_row + 2  

        table_data = [
            ["", "PM Basique", "", "Niveau", "N° de Phase", "Date", "Préparé par", "Timbre"],
            ["Note:", "Les cases colorées sont des PM optionnelles", "", "", "", "", "", ""],
            ["", "🌀 : ", "A Insérer", "", "", "", "", ""],
            ["", "Ø : ", "A Ne pas insérer", "", "", "", "", ""],
            ["", "⊖ : ", "Déjà inséré", "", "", "", "", ""]
        ]

        # Définir les bordures
        border = Border(
            top=Side(style='thin'),
            bottom=Side(style='thin'),
            left=Side(style='thin'),
            right=Side(style='thin')
        )

        for row_index, row_data in enumerate(table_data, start=row_start):
            for col_index, cell_value in enumerate(row_data, start=1):
                ws.cell(row=row_index, column=col_index, value=cell_value)
                ws.cell(row=row_index, column=col_index).border = border
                ws.cell(row=row_index, column=col_index).font = Font(size=12, bold=True)



            
    # # Appliquer les bordures        
    # for row in ws.iter_rows(min_row=row_start, max_row=row_start + len(table_data) - 1, min_col=1, max_col=len(table_data[0])):
    #     for cell in row:
    #         cell.border = border
            
    # print("Tableau ajouté avec succès !")
    wb.save(path_output_excel)
    wb.close()

add_last_table(path_output_excel)



In [37]:
print(type(add_last_table))

<class 'function'>


In [15]:
# import xlwings as xw
# import pandas as pd
# import numpy as np


    
# with pd.ExcelWriter('multi_feuilles.xlsx', engine='xlsxwriter') as writer:      

#   for sheet in all_sheets:
#       if pd.isna(sheet):
#           continue
#       sheet_name = wb.sheets.add(name=sheet)
      
#       sps_data = df_input[df_input['SPS'] == sheet]
      
#       arrays = [np.array(sps_data['Production Module MA15']),
#                 np.array(sps_data['Step']),
#               #   np.array(sps_data['Component Name']),
#                 np.array(sps_data['SAP NO MA15']),
#                 np.array(sps_data['Note']),
#                 np.array(sps_data['CS'])          
#                         ]
#       index = pd.MultiIndex.from_arrays(arrays, names=('Production Module MA15', 'Step', 'SAP NO MA15', 'Note', 'CS'))
      
#       df = pd.DataFrame({'Component Name': sps_data['Component Name'].values},
#                       index=index)
      
#       # sheet_name.range('A1').options(index=True).value = df
#       df.to_excel(writer, sheet_name=sheet, index=True)

# wb.sheets[0].delete()

In [16]:
# import xlwings as xw
# import pandas as pd
# import numpy as np

# wb = xw.Book()
# sheet = wb.sheets[0]
    
# sps_data = df_input[df_input['SPS'] == "SA36-A"]
    
# arrays = [np.array(sps_data['Production Module MA15']),
#               np.array(sps_data['Step'])         
#                        ]
# index = pd.MultiIndex.from_arrays(arrays, names=('Production Module MA15', 'Step'))
    
# df = pd.DataFrame({'Component Name': sps_data['Component Name'].values},
#                      index=index)
    
# sheet.range('A1').value = df



In [17]:
# import openpyxl
# from openpyxl.styles import Border, Side

# # Charger le fichier Excel
# workbook = openpyxl.load_workbook('schema.xlsx')
# sheet = workbook.active  # Sélectionner la première feuille (ou spécifiez le nom de la feuille)

# # Trouver la dernière ligne
# last_row = sheet.max_row +2

# # Les données à insérer
# tableau = [
#     ["PM Basique", "Note", "Les cases colorées sont des PM optionnelles", "", "", "", "", "", ""],
#     ["", "🌀 :", "A Insérer", "", "", "", "", "", ""],
#     ["", "Ø :", "A Ne pas insérer", "", "", "", "", "", ""],
#     ["", "O :", "Déjà inséré", "", "", "", "", "", ""]
# ]

# # Insérer le tableau à la ligne suivante
# for i, row in enumerate(tableau, start=last_row + 2):
#     for j, value in enumerate(row):
#         sheet.cell(row=i, column=j+1, value=value)

# # Définir la taille des colonnes
# column_widths = [15, 35, 50, 10, 10, 10, 15, 15, 15]
# for col_num, width in enumerate(column_widths, start=1):
#     sheet.column_dimensions[openpyxl.utils.get_column_letter(col_num)].width = width

# # Définir les bordures
# border = Border(
#     top=Side(style='thin'),
#     bottom=Side(style='thin'),
#     left=Side(style='thin'),
#     right=Side(style='thin')
# )

# # Appliquer les bordures à toutes les cellules
# for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
#     for cell in row:
#         cell.border = border

# # Sauvegarder le fichier
# workbook.save('votre_fichier_modifié.xlsx')
